In [32]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import os

## Manufacturing Parts Process

Analyze the manufacturing_parts table and determine whether the manufacturing process is performing within acceptable control limits:.

* Create an alert that flags whether the height of a product is within the control limits for each operator using the formulas provided in the workbook.
    - The final query should return the following fields: operator, row_number, height, avg_height, stddev_height, ucl, lcl, alert, and be ordered by the item_no.
    - The alert column will be your boolean flag.
    - Use a window function of length 5 to calculate the control limits, considering rows up to and including the current row; incomplete windows should be excluded from the final query output. Save this DataFrame as alerts.

<br>

Manufacturing processes for any product is like putting together a puzzle. Products are pieced together step by step, and keeping a close eye on the process is important. You're supporting a team that wants to improve how they monitor and control a manufacturing process. The goal is to implement a more methodical approach known as statistical process control (SPC). SPC is an established strategy that uses data to determine whether the process works well. Processes are only adjusted if measurements fall outside of an acceptable range. 

This acceptable range is defined by an upper control limit (UCL) and a lower control limit (LCL), the formulas for which are:

$ucl = avg\_height + 3 * \frac{stddev\_height}{\sqrt{5}}$

$lcl = avg\_height - 3 * \frac{stddev\_height}{\sqrt{5}}$

The UCL defines the highest acceptable height for the parts, while the LCL defines the lowest acceptable height for the parts. Ideally, parts should fall between the two limits.

Using SQL window functions and nested queries, you'll analyze historical manufacturing data to define this acceptable range and identify any points in the process that fall outside of the range and therefore require adjustments. This will ensure a smooth running manufacturing process consistently making high-quality products.

<br>

The data is available in the `manufacturing_parts` table which has the following fields:
- `item_no`: the item number
- `length`: the length of the item made
- `width`: the width of the item made
- `height`: the height of the item made
- `operator`: the operating machine

In [33]:
# Load CSV into DataFrame
manu_parts_df = pd.read_csv('../data_raw/manu_parts.csv')

# Create SQLite database
conn = sqlite3.connect("../data_raw/manu_parts.db")
manu_parts_df.to_sql("manu_parts", conn, if_exists="replace", index=False)
conn.close()
print("Data loaded into SQLite database successfully.")

engine = create_engine("sqlite:///../data_raw/manu_parts.db")

query = """ SELECT * FROM manu_parts; """
manu_parts = pd.read_sql_query(query, engine)
manu_parts

Data loaded into SQLite database successfully.


,item_no,length,width,height,operator
0,1,102.67,49.53,19.69,Op-1
1,2,102.50,51.42,19.63,Op-1
2,3,95.37,52.25,21.51,Op-1
3,4,94.77,49.24,18.60,Op-1
4,5,104.26,47.90,19.46,Op-1
...,...,...,...,...,...
495,496,101.24,49.03,20.96,Op-20
496,497,98.37,52.12,19.68,Op-20
497,498,96.49,48.78,19.19,Op-20
498,499,94.16,48.39,21.60,Op-20


In [34]:
# # SQLite doesn't support STDDEV, so we need to calculate it manually

# query = """ 
# -- Flag whether the height of a product is within the control limits
# SELECT
# 	b.*,
# 	CASE
# 		WHEN 
# 			b.height NOT BETWEEN b.lcl AND b.ucl
# 		THEN TRUE
# 		ELSE FALSE
# 	END as alert
# FROM (
# 	SELECT
# 		a.*, 
# 		a.avg_height + 3*a.stddev_height/SQRT(5) AS ucl, 
# 		a.avg_height - 3*a.stddev_height/SQRT(5) AS lcl  
# 	FROM (
# 		SELECT 
# 			operator,
# 			ROW_NUMBER() OVER w AS row_number, 
# 			height, 
# 			AVG(height) OVER w AS avg_height, 
# 			STDDEV(height) OVER w AS stddev_height
# 		FROM manu_parts 
# 		WINDOW w AS (
# 			PARTITION BY operator 
# 			ORDER BY item_no 
# 			ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
# 		)
# 	) AS a
# 	WHERE a.row_number >= 5
# ) AS b; """

# manu_parts = pd.read_sql_query(query, engine)
# manu_parts

## Motorcycle Parts Sales

Find out how much Wholesale net revenue each product_line generated per month per warehouse in the dataset.

* The query should be saved as revenue_by_product_line using the SQL cell provided, and contain the following:
    - product_line,
    - month: displayed as 'June', 'July', and 'August',
    - warehouse, and
    - net_revenue: the sum of total minus the sum of payment_fee.
* The results should be sorted by product_line and month, followed by net_revenue in descending order.

<br>

You're working for a company that sells motorcycle parts, and they've asked for some help in analyzing their sales data!

They operate three warehouses in the area, selling both retail and wholesale. They offer a variety of parts and accept credit cards, cash, and bank transfer as payment methods. However, each payment type incurs a different fee.

The board of directors wants to gain a better understanding of wholesale revenue by product line, and how this varies month-to-month and across warehouses. You have been tasked with calculating net revenue for each product line and grouping results by month and warehouse. The results should be filtered so that only `"Wholesale"` orders are included.

<br>

`motorcycle_sales`

| Column | Data type | Description |
|--------|-----------|-------------|
| `order_number` | `VARCHAR` | Unique order number. |
| `date` | `DATE` | Date of the order, from June to August 2021. |
| `warehouse` | `VARCHAR` | The warehouse that the order was made from&mdash; `North`, `Central`, or `West`. |
| `client_type` | `VARCHAR` | Whether the order was `Retail` or `Wholesale`. |
| `product_line` | `VARCHAR` | Type of product ordered. |
| `quantity` | `INT` | Number of products ordered. | 
| `unit_price` | `FLOAT` | Price per product (dollars). |
| `total` | `FLOAT` | Total price of the order (dollars). |
| `payment` | `VARCHAR` | Payment method&mdash;`Credit card`, `Transfer`, or `Cash`. |
| `payment_fee` | `FLOAT` | Percentage of `total` charged as a result of the `payment` method. |


Your query output should be presented in the following format:

| `product_line` | `month` | `warehouse` |	`net_revenue` |
|----------------|-----------|----------------------------|--------------|
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_two | --- | --- | --- |
| ... | ... | ... | ... |

In [35]:
# Load CSV into DataFrame
motorcycle_sales_df = pd.read_csv('../data_raw/motorcycle_sales.csv')

# Create SQLite database
conn = sqlite3.connect("../data_raw/motorcycle_sales.db")
motorcycle_sales_df.to_sql("motorcycle_sales", conn, if_exists="replace", index=False)
conn.close()
print("Data loaded into SQLite database successfully.")

engine = create_engine("sqlite:///../data_raw/motorcycle_sales.db")

query = """ SELECT * FROM motorcycle_sales; """
motorcycle_sales = pd.read_sql_query(query, engine)
motorcycle_sales

Data loaded into SQLite database successfully.


,index,order_number,date,warehouse,client_type,product_line,quantity,unit_price,total,payment,payment_fee
0,0,N1,2021-06-01T00:00:00.000,North,Retail,Braking system,9,19.29,173.61,Cash,0.00
1,1,N2,2021-06-01T00:00:00.000,North,Retail,Suspension & traction,8,32.93,263.45,Credit card,0.03
2,2,N3,2021-06-01T00:00:00.000,North,Wholesale,Frame & body,16,37.84,605.44,Transfer,0.01
3,3,N4,2021-06-01T00:00:00.000,North,Wholesale,Suspension & traction,40,37.37,1494.80,Transfer,0.01
4,4,N5,2021-06-01T00:00:00.000,North,Retail,Frame & body,6,45.44,272.61,Credit card,0.03
...,...,...,...,...,...,...,...,...,...,...,...
995,995,W176,2021-08-27T00:00:00.000,West,Retail,Electrical system,4,21.47,85.89,Credit card,0.03
996,996,W177,2021-08-28T00:00:00.000,West,Wholesale,Miscellaneous,32,25.42,813.44,Transfer,0.01
997,997,W178,2021-08-28T00:00:00.000,West,Retail,Electrical system,6,28.37,170.21,Credit card,0.03
998,998,W179,2021-08-28T00:00:00.000,West,Wholesale,Braking system,32,10.03,320.96,Transfer,0.01


In [36]:
query = """ 
SELECT product_line,
    CASE 
        WHEN strftime('%m', date) = '06' THEN 'June'
        WHEN strftime('%m', date) = '07' THEN 'July'
        WHEN strftime('%m', date) = '08' THEN 'August'
    END as month,
    warehouse,
    SUM(total) - SUM(payment_fee) AS net_revenue
FROM motorcycle_sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, warehouse, month
ORDER BY product_line, month, net_revenue DESC
"""
motorcycle_sales = pd.read_sql_query(query, engine)
motorcycle_sales

,product_line,month,warehouse,net_revenue
0,Braking system,August,Central,3039.41
1,Braking system,August,West,2500.67
2,Braking system,August,North,1770.84
3,Braking system,July,Central,3778.65
4,Braking system,July,West,3060.93
5,Braking system,July,North,2594.44
6,Braking system,June,Central,3684.89
7,Braking system,June,North,1487.77
8,Braking system,June,West,1212.75
9,Electrical system,August,North,4721.12


## Electric Car Charging Habits

Write three SQL queries to answer the following questions:

1. Find the number of unique individuals that use each garage’s shared charging stations. The output should contain two columns: garage_id and num_unique_users. Sort your results by the number of unique users from highest to lowest. Save the result as unique_users_per_garage.

2. Find the top 10 most popular charging start times (by weekday and start hour) for sessions that use shared charging stations. Your result should contain three columns: weekdays_plugin, start_plugin_hour, and a column named num_charging_sessions containing the number of plugins on that weekday at that hour. Sort your results from the most to the least number of sessions. Save the result as most_popular_shared_start_times.

As electronic vehicles (EVs) become more popular, there is an increasing need for access to charging stations, also known as ports. To that end, many modern apartment buildings have begun retrofitting their parking garages to include shared charging stations. A charging station is shared if it is accessible by anyone in the building.

But with increasing demand comes competition for these ports — nothing is more frustrating than coming home to find no charging stations available! In this project, you will use a dataset to help apartment building managers better understand their tenants’ EV charging habits.

<br>

`charging_sessions`

| Column | Definition | Data type |
|-|-|-|
|`garage_id`| Identifier for the garage/building|`VARCHAR`|
|`user_id` | Identifier for the individual user|`VARCHAR`|
|`user_type`|Indicating whether the station is `Shared` or `Private`| `VARCHAR` |
|`start_plugin`|The date and time the session started |`DATETIME`|
|`start_plugin_hour`|The hour (in military time) that the session started | `NUMERIC`|
|`end_plugout`|The date and time the session ended | `DATETIME` |
|`end_plugout_hour`|The hour (in military time) that the session ended | `NUMERIC`|
|`duration_hours`| The length of the session, in hours|`NUMERIC`|
|`el_kwh`| Amount of electricity used (in Kilowatt hours)|`NUMERIC`|
|`month_plugin`| The month that the session started |`VARCHAR`|
|`weekdays_plugin`| The day of the week that the session started|`VARCHAR`|


In [37]:
# Load CSV into DataFrame
charging_sessions_df = pd.read_csv('../data_raw/charging_sessions.csv')

# Create SQLite database
conn = sqlite3.connect("../data_raw/charging_sessions.db")
charging_sessions_df.to_sql("charging_sessions", conn, if_exists="replace", index=False)
conn.close()
print("Data loaded into SQLite database successfully.")

engine = create_engine("sqlite:///../data_raw/charging_sessions.db")

query = """ SELECT * FROM charging_sessions; """
charging_sessions = pd.read_sql_query(query, engine)
charging_sessions

Data loaded into SQLite database successfully.


,index,garage_id,user_id,user_type,start_plugin,start_plugin_hour,end_plugout,end_plugout_hour,el_kwh,duration_hours,month_plugin,weekdays_plugin
0,0,AdO3,AdO3-4,Private,2018-12-21T10:20:00.000,10,2018-12-21T10:23:00.000,10.0,0.30,0.050000,Dec,Friday
1,1,AdO3,AdO3-4,Private,2018-12-21T10:24:00.000,10,2018-12-21T10:32:00.000,10.0,0.87,0.136667,Dec,Friday
2,2,AdO3,AdO3-4,Private,2018-12-21T11:33:00.000,11,2018-12-21T19:46:00.000,19.0,29.87,8.216389,Dec,Friday
3,3,AdO3,AdO3-2,Private,2018-12-22T16:15:00.000,16,2018-12-23T16:40:00.000,16.0,15.56,24.419722,Dec,Saturday
4,4,AdO3,AdO3-2,Private,2018-12-24T22:03:00.000,22,2018-12-24T23:02:00.000,23.0,3.62,0.970556,Dec,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...
6873,6873,UT15,UT15-2,Private,2020-01-31T17:57:00.000,17,2020-01-31T22:29:00.000,22.0,13.43,4.530000,Jan,Friday
6874,6874,AsO10,AsO10-1,Private,2020-01-31T18:32:00.000,18,2020-01-31T21:43:00.000,21.0,9.10,3.182222,Jan,Friday
6875,6875,Bl2,Bl2-1,Private,2020-01-31T19:25:00.000,19,2020-01-31T20:37:00.000,20.0,4.28,1.203889,Jan,Friday
6876,6876,UT7,UT7-1,Private,2020-01-31T19:27:00.000,19,2020-01-31T19:48:00.000,19.0,2.31,0.352500,Jan,Friday


In [38]:
query = """ 
-- unique_users_per_garage

SELECT garage_id, 
	COUNT(DISTINCT user_id) AS num_unique_users
FROM charging_sessions
WHERE user_type = 'Shared'
GROUP BY garage_id
ORDER BY num_unique_users DESC; """
charging_sessions = pd.read_sql_query(query, engine)
charging_sessions

,garage_id,num_unique_users
0,Bl2,18
1,AsO2,17
2,UT9,16
3,AdO3,3
4,SR2,2
5,MS1,2
6,Ris,1
7,AdA1,1


In [39]:
query = """ 
-- most_popular_shared_start_times

SELECT weekdays_plugin,
	start_plugin_hour, 
	COUNT(*) AS num_charging_sessions
FROM charging_sessions
WHERE user_type = 'Shared'
GROUP BY weekdays_plugin,
	start_plugin_hour
ORDER BY num_charging_sessions DESC
LIMIT 10;
"""
charging_sessions = pd.read_sql_query(query, engine)
charging_sessions

,weekdays_plugin,start_plugin_hour,num_charging_sessions
0,Sunday,17,30
1,Friday,15,28
2,Thursday,16,26
3,Thursday,19,26
4,Sunday,15,25
5,Sunday,18,25
6,Wednesday,19,25
7,Friday,16,24
8,Monday,15,24
9,Saturday,17,23


In [40]:
query = """ 
-- long_duration_shared_users

SELECT user_id, 
	AVG(duration_hours) as avg_charging_duration
FROM charging_sessions 
WHERE user_type = 'Shared' 
GROUP BY user_id
HAVING AVG(duration_hours) > 10
ORDER BY AVG(duration_hours) DESC;
"""
charging_sessions = pd.read_sql_query(query, engine)
charging_sessions

,user_id,avg_charging_duration
0,Share-9,16.845833
1,Share-17,12.894556
2,Share-25,12.214475
3,Share-18,12.088807
4,Share-8,11.550431
5,AdO3-1,10.369387


## Car Insurance Claims

* Identify the single feature of the data that is the best predictor of whether a customer will put in a claim (the "outcome" column), excluding the "id" column.
* Store as a DataFrame called best_feature_df, containing columns named "best_feature" and "best_accuracy" with the name of the feature with the highest accuracy, and the respective accuracy score.

<br>

Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) 

Knowing all of this, Car Insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

<br>

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [41]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Read in dataset
cars = pd.read_csv("../data_raw/car_insurance.csv")

# Check for missing values
cars.info()

# Fill missing values with the mean
cars["credit_score"].fillna(cars["credit_score"].mean(), inplace=True)
cars["annual_mileage"].fillna(cars["annual_mileage"].mean(), inplace=True)

# Empty list to store model results
models = []

# Feature columns
features = cars.drop(columns=["id", "outcome"]).columns

# Loop through features
for col in features:
    # Create a model
    model = logit(f"outcome ~ {col}", data=cars).fit()
    # Add each model to the models list
    models.append(model)

# Empty list to store accuracies
accuracies = []

# Loop through models
for feature in range(0, len(models)):
    # Compute the confusion matrix
    conf_matrix = models[feature].pred_table()
    # True negatives
    tn = conf_matrix[0,0]
    # True positives
    tp = conf_matrix[1,1]
    # False negatives
    fn = conf_matrix[1,0]
    # False positives
    fp = conf_matrix[0,1]
    # Compute accuracy
    acc = (tn + tp) / (tn + fn + fp + tp)
    accuracies.append(acc)

# Find the feature with the largest accuracy
best_feature = features[accuracies.index(max(accuracies))]

# Create best_feature_df
best_feature_df = pd.DataFrame({"best_feature": best_feature,
                                "best_accuracy": max(accuracies)},
                                index=[0])
best_feature_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       9043 non-null   float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

/var/folders/ln/y2zb_fq101s1gcvy6lnz_0tw0000gn/T/ipykernel_5836/857789780.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cars["credit_score"].fillna(cars["credit_score"].mean(), inplace=True)
/var/folders/ln/y2zb_fq101s1gcvy6lnz_0tw0000gn/T/ipykernel_5836/857789780.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are

Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595431
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.617345
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.605716
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621700
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.558922
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.598699
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.549220
         Iterations 7


,best_feature,best_accuracy
0,driving_experience,0.7771
